每个RGB输入图像具有  3×h×w  的形状。我们将这个大小为  3  的轴称为 通道（channel） 维度

## 1、多输入通道

当输入包含多个通道时，需要构造一个与输入数据具有相同输入通道数目的卷积核，以便与输入数据进行互相关运算。假设输入的通道数为  ci ，那么卷积核的输入通道数也需要为  ci  。

In [1]:
import torch
from d2l import torch as d2l
import sys
sys.path.append('../../CommonFunctions/tools/')
import common_tools

In [27]:
def corr2d_multi_in(X, K):
    # 先遍历 “X” 和 “K” 的第0个维度（通道维度），再把它们加在一起
    return sum(common_tools.corr2d(x, k) for x, k in zip(X, K))

In [28]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

In [29]:
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

In [30]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 2、多输出通道

在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。

直观地说，我们可以将每个通道看作是对不同特征的响应。

而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

卷积核的形状是  co×ci×kh×kw 

In [31]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [32]:
K = torch.stack((K, K +1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [33]:
K

tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]])

In [34]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 3、1x1 卷积层
因为使用了最小窗口， 1×1  卷积失去了卷积层的特有能力——在高度和宽度维度上，识别相邻元素间相互作用的能力。 而  1×1  卷积的唯一计算发生在通道上。

我们可以将  1×1  卷积层看作是在每个像素位置应用的全连接层，以  ci  个输入值转换为  co  个输出值。 因为这仍然是一个卷积层，所以跨像素的权重是一致的。 同时， 1×1  卷积层需要的权重维度为  co×ci  ，再额外加上一个偏置。

使用全连接层实现  1×1  卷积。 请注意，我们需要对输入和输出的数据形状进行微调。

In [35]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [39]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [40]:
X.shape, K.shape

(torch.Size([3, 3, 3]), torch.Size([2, 3, 1, 1]))

In [43]:
# 当执行  1×1  卷积运算时，上述函数相当于先前实现的互相关函数corr2d_multi_in_out。
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6